In [134]:
import pandas as pd

from datetime import datetime, timedelta
import pyodbc
import pymssql

In [135]:
server = 'tcp:lw-azuresql-1.database.windows.net'
database = 'LesewertMandantenDB_SDZ'
username = 'Mehrwertmacher'
password = 'fW18ObHgb8qVhlQUsNpx'
driver= '{ODBC Driver 17 for SQL Server}'

# Abfrge Geschlecht

In [26]:
sql_kum = '''
declare @Von DATE = '2021-04-12',
@Bis DATE = '2021-06-05', 
@Wellestart INTEGER = 4, 
@Welleende INTEGER = 8, 
@Ausgabestart INTEGER = 1, 
@Ausgabeende INTEGER = 5;


WITH complete AS (
SELECT ts.TeilartikelId, ts.TeilartikelveroeffentlichungId
FROM TeilartikelStatistiken ts
JOIN vw_Artikellesewerte_Kumuliert_AktiveWellen_Vorberechnet kum
ON kum.ArtikelId = ts.TeilartikelVeroeffentlichungId
WHERE kum.Erscheinungsdatum BETWEEN @Von AND @Bis
AND kum.Wellennummer BETWEEN @Wellestart AND @Welleende),


male AS (
SELECT Lesewert LW_m, Blickwert BW_m, Durchlesewert DW_m, TeilartikelId, TeilartikelVeroeffentlichungId
FROM TeilartikelStatistiken 
WHERE Geschlecht = 'm'
AND Kumuliert = 1), 
female AS (
SELECT Lesewert LW_w, Blickwert BW_w, Durchlesewert DW_w, TeilartikelId, TeilartikelVeroeffentlichungId
FROM TeilartikelStatistiken 
WHERE Geschlecht = 'w'
AND Kumuliert = 1), 


age_40_59 AS (
SELECT Lesewert LW_40bis59, Blickwert BW_40bis59, Durchlesewert DW_40bis59, TeilartikelId, TeilartikelVeroeffentlichungId
FROM TeilartikelStatistiken
WHERE AltersgruppenId = 3
AND Kumuliert = 1), 

age_60_79 AS (
SELECT Lesewert LW_60bis79, Blickwert BW_60bis79, Durchlesewert DW_60bis79, TeilartikelId, TeilartikelVeroeffentlichungId
FROM TeilartikelStatistiken
WHERE AltersgruppenId = 4
AND Kumuliert = 1),
platzierung AS (
SELECT Platzierung, TeilartikelId, TeilartikelVeroeffentlichungId
FROM TeilartikelStatistiken
WHERE Kumuliert = 1
AND Geschlecht IS NULL 
AND AltersgruppenId IS NULL
AND OrtsgruppenId IS NULL
)




SELECT DISTINCT c.TeilartikelVeroeffentlichungId, c.TeilartikelId, p.Platzierung, f.LW_w, f.BW_w, f.DW_w, m.LW_m, m.BW_m, m.DW_m, 
age1.LW_40bis59, age1.BW_40bis59, age1.DW_40bis59, 
age2.LW_60bis79, age2.BW_60bis79, age2.DW_60bis79


  
FROM complete c
LEFT JOIN male m
ON c.TeilartikelVeroeffentlichungId = m.TeilartikelVeroeffentlichungId
LEFT JOIN female f
ON c.TeilartikelVeroeffentlichungId = f.TeilartikelVeroeffentlichungId

LEFT JOIN age_40_59 age1
ON c.TeilartikelVeroeffentlichungId = age1.TeilartikelVeroeffentlichungId
LEFT JOIN age_60_79 age2
ON c.TeilartikelVeroeffentlichungId = age2.TeilartikelVeroeffentlichungId

LEFT JOIN platzierung p
ON c.TeilartikelVeroeffentlichungId = p.TeilartikelVeroeffentlichungId
ORDER BY c.TeilartikelVeroeffentlichungId



'''

In [27]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [28]:
df_kum = pd.read_sql(sql_kum, cnxn)


In [21]:
df_nichtkum.to_csv("./Daten/geschlecht_nichtkum.csv", index=False, decimal=",")

In [29]:
df_kum

,TeilartikelVeroeffentlichungId,TeilartikelId,Platzierung,LW_w,BW_w,DW_w,LW_m,BW_m,DW_m,LW_40bis59,BW_40bis59,DW_40bis59,LW_60bis79,BW_60bis79,DW_60bis79
0,15904,4996,117,2.020906,2.857143,70.731709,1.850385,3.030303,61.062718,0.896700,2.941176,30.487804,1.382114,2.222222,62.195122
1,15905,4997,22,22.698380,32.592593,69.642758,28.291848,40.243902,70.300955,31.025763,42.857143,72.393447,23.609429,34.523810,68.385934
2,15907,5001,40,11.781824,19.259259,61.174854,23.250857,32.113821,72.401401,22.662862,30.952381,73.218477,17.461048,32.142857,54.323260
3,15909,5002,30,20.343800,25.892857,78.569160,23.315128,27.669903,84.261691,13.542252,20.000000,67.711258,18.521660,24.657534,75.115620
4,15910,5000,55,16.888311,28.148148,59.997948,11.362304,17.479675,65.002946,33.922770,47.619048,71.237816,8.368143,15.476190,54.071076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40190,61520,19307,193,1.378246,1.388889,99.233719,0.799169,6.153846,12.986490,NaN,NaN,NaN,0.019620,5.405405,0.362976
40191,61521,19309,198,NaN,NaN,NaN,1.292884,4.615385,28.012482,NaN,NaN,NaN,2.662480,2.702703,98.511765
40192,61522,19276,128,7.907650,18.055556,43.796218,7.943694,10.769231,73.762869,0.973703,18.181818,5.355364,11.992963,18.918919,63.391375
40193,61523,19298,158,5.137060,9.722222,52.838335,4.035573,6.153846,65.578065,5.738078,9.090909,63.118858,6.993049,8.108108,86.247604


In [30]:
df_kum.to_csv("./Daten/geschlecht_kum.csv", index=False, decimal=",")

# Abfrage nach Wochenendleser vs. Wochenleser

# Abfrage Kumulierte Texte mit NB 

In [6]:
sql_kum = '''
declare @Von DATE = '2021-04-12',
@Bis DATE = '2021-06-05', 
@Wellestart INTEGER = 4, 
@Welleende INTEGER = 8, 
@Ausgabestart INTEGER = 1, 
@Ausgabeende INTEGER = 5;



SELECT * 
FROM vw_Artikellesewerte_Kumuliert_AktiveWellen_Vorberechnet kum
WHERE kum.Erscheinungsdatum BETWEEN @Von AND @Bis
AND kum.Wellennummer BETWEEN @Wellestart AND @Welleende

'''

In [7]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [8]:
df_kum = pd.read_sql(sql_kum, cnxn)

In [9]:
df_kum.to_csv("kum_mit_nb.csv", index=False)

# Alle Artikeltexte

In [11]:
sql_all = '''
declare @Von DATE = '2021-04-12',
@Bis DATE = '2021-06-05', 
@Wellestart INTEGER = 4, 
@Welleende INTEGER = 8, 
@Ausgabestart INTEGER = 1, 
@Ausgabeende INTEGER = 5;



SELECT * 
FROM Teilartikelinhalte te

'''

In [12]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [13]:
df_all = pd.read_sql(sql_all, cnxn)

In [14]:
df_all.shape

(19658, 14)

In [15]:
df_all

,Id,TeilartikelId,Dachzeilentext,Ueberschrifttext,Untertiteltext,Artikeltext,ArtikeltextHtml,Vorspanntext,Unterschrifttext,Bildunterschrifttext,Importdatum,VersionId,RowVersion,Wortanzahl
0,1,1,,Gewinnzahlen,,Gewinnzahlen vom Wochenende\r\nLotto (20.03.):...,,,,,2021-03-22 13:59:51.917,None,b'\x00\x00\x00\x00\x00\x00\xbf\x12',None
1,2,3,,Glühende Bäche,,Ein Hubschrauber überfliegt am Samstag den Fag...,,,,,2021-03-22 14:04:29.337,None,b'\x00\x00\x00\x00\x00\x00\xcfA',None
2,3,4,,Türkei schwächt den Schutz von Frauen,Präsident Erdoğan tritt aus der Istanbul-Konve...,Istanbul – Mit dem Austritt aus der Istanbul-K...,,,,,2021-03-22 14:04:30.353,None,b'\x00\x00\x00\x00\x00\x00\xcfX',None
3,4,2,,HEUTE,,Meinung\r\nDie Bundesregierung verpasst\r\nein...,,,,Foto: Hector Vivas/Getty,2021-03-22 14:04:24.517,None,b'\x00\x00\x00\x00\x00\x00\xce\xaf',None
4,5,5,,SZ App,,Die SZ gibt es als App\r\nfür Tablet und Smart...,,,,,2021-03-22 14:04:25.620,None,b'\x00\x00\x00\x00\x00\x00\xce\xd2',None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19653,19683,19668,,ohne_Titel,,Leserbriefe stellen keine redaktionelle Meinun...,,,,,2021-06-07 03:04:08.390,None,"b""\x00\x00\x00\x00\x00\xdcW'""",None
19654,19684,19669,,Motorradfahrer von Auto erfasst,,Gauting – Ein 29-jähriger Motorradfahrer aus M...,,,,,2021-06-07 03:04:08.423,None,b'\x00\x00\x00\x00\x00\xdcW*',None
19655,19685,19670,,Mörderisches Puzzle,Die Gautinger Autorin Anna Schneider ist mit d...,Gauting – Alexa Jahn hat ihren Dienst in der K...,,,,"Der zweite Fall ist schon fertig, der dritte i...",2021-06-07 03:04:08.487,None,"b'\x00\x00\x00\x00\x00\xdcW,'",None
19656,19686,19671,,LESERBRIEFE,,,,,,,2021-06-07 03:04:08.590,None,b'\x00\x00\x00\x00\x00\xdcW8',None


# Nichtkum-Artikel 

In [9]:
sql_nichtkum = '''
declare @Von DATE = '2021-04-12',
@Bis DATE = '2021-06-05', 
@Wellestart INTEGER = 4, 
@Welleende INTEGER = 8, 
@Ausgabestart INTEGER = 1, 
@Ausgabeende INTEGER = 5;



SELECT * 
FROM vw_Artikellesewerte_Kumuliert_AktiveWellen_Vorberechnet kum
WHERE kum.Erscheinungsdatum BETWEEN @Von AND @Bis
AND kum.Wellennummer BETWEEN @Wellestart AND @Welleende

'''

In [10]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [11]:
df_nichtkum = pd.read_sql(sql_nichtkum, cnxn)

In [12]:
df_nichtkum.shape

(40195, 26)

In [16]:
df_nk = df_nichtkum.copy()

In [17]:
df_nk = df_nk[df_nk["NB"]!="NB"]
df_nk.shape

(36983, 26)

In [18]:
df_nk

,SplitId,ArtikelId,Ueberschrifttext,ErscheinungsId,AusgabeId,Erscheinungsdatum,Seitennummer,Ressortname,UnterRessortname,Ausgabeleser (Erscheinung),...,Platzierungen,Darstellungsformen,Beschreibung,Handlungsorte,Akteure,Themen,Ausgabename,Wellennummer,offen/codiert,NB
1,4997,15905,Einfach zu wenig Boxen,81,1,2021-04-12,15,R2466,R2466,381,...,AA,BF,Analyse,"Köln, Hamburg, Australien, Los Angeles, Ägypte...","Ever Given, Hapag-Lloyd, Rainer Horn","Business und Finanzen, Industrie und Branchen ...",HF3,4,codiert,None
2,5001,15907,Bittere Printen,81,1,2021-04-12,15,R2466,R2466,381,...,SK,BF,None,"Deutschland, Washington, Brüssel, Vereinigte S...","Joe Biden, Gabriel Felbermayr, Donald Trump, A...","Business allgemein, Business und Finanzen, Ges...",HF3,4,codiert,None
3,5002,15909,Freudige Erwartung,81,1,2021-04-12,2,R2461,R2461,318,...,TS1,BF,Bericht,"Deutschland, Corona","Markus Söder, Armin Laschet, Dorothee Bär, Tho...","Politik, Wahlen, Politische Institutionen & Er...",HF3,4,codiert,None
4,5000,15910,Trauer im Königreich,81,1,2021-04-12,1,R2441,R2441,381,...,ÜA,BN,None,"London, Westminster, Windsor, Elizabeth","Elizabeth II., Piccadilly Circus","Popkultur, Promi-Familien, Society/Promis/Klat...",HF3,4,codiert,None
5,5004,15911,Wie Hund und Katz,81,1,2021-04-12,16,R2466,R2466,381,...,TS1,KM,Nach Corona,Deutschland,"Zooplus, Thomas Schröder","Haustiere und freilebende Tiere, Haustiere, Ti...",HF3,4,codiert,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40190,19176,61251,Aufgemerkt,299,4,2021-06-05,40,R2411,R2411,329,...,TS1,NA,None,None,"Netflix, Barack Obama, Amanda Gorman, Michelle...","Filme, Horrorfilme, Politische Institutionen &...",1MG_Ebersberg,8,codiert,None
40191,19420,61252,KFC kommt nach Parsdorf,299,4,2021-06-05,77,R2339,R2339,48,...,ZA,BF,None,"Deutschland, Vereinigte Staaten von Amerika",Christlich-Soziale Union in Bayern,"Business allgemein, Business und Finanzen, Ind...",1MG_Ebersberg,8,codiert,None
40192,19318,61254,OP-Masken für Schüler,299,4,2021-06-05,79,R4124,R4124,202,...,TS1,NA,None,München,None,"Bildung, Fern- und Online-Unterricht, Grundsch...",1MG_Ebersberg,8,codiert,None
40193,19419,61255,Zorneding,299,4,2021-06-05,77,R2339,R2339,48,...,TS1,"AK, NA",None,None,None,"Haustiere und freilebende Tiere, Veranstaltungen",1MG_Ebersberg,8,codiert,None


In [20]:
df_nichtkum.to_csv("nichtkum_mit_nb.csv", index=False)

# LW für Wochenend-Leser heraussuchen

In [37]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [38]:
sql_all = '''


SELECT * 
FROM TeilnehmerDurchlesewerte




'''

sql_scan = '''

SELECT * 
FROM ScanAuswertungen

'''


In [31]:
df_all = pd.read_sql(sql_all, cnxn)

In [178]:
df_all.to_pickle("df_scans.pkl")

In [39]:
df_scan = pd.read_sql(sql_scan, cnxn)

In [174]:
df_scan.WellenteilnahmenId.nunique()

618

In [175]:
df_scan.to_csv("df_scanauswertung.csv", index=False)

In [46]:
df_scan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728696 entries, 0 to 728695
Data columns (total 19 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Id                              728696 non-null  int64         
 1   Dateiname                       728696 non-null  object        
 2   Dateidatum                      728696 non-null  datetime64[ns]
 3   Xmlzeile                        728696 non-null  bool          
 4   Scanzeile                       728696 non-null  object        
 5   Suchzeile                       633922 non-null  object        
 6   Treffer                         728696 non-null  bool          
 7   Artikelzeile                    0 non-null       object        
 8   TeilnehmerDurchlesewert         600292 non-null  float64       
 9   Bewertung                       728696 non-null  object        
 10  Erfassungsdatum                 728696 non-null  object 

In [50]:
df_scan.Erscheinungsdatum

0         2021-03-15
1         2021-03-15
2         2021-03-15
3         2021-03-15
4         2021-03-15
             ...    
728691    2021-07-08
728692    2021-07-08
728693    2021-07-08
728694    2021-07-08
728695    2021-07-08
Name: Erscheinungsdatum, Length: 728696, dtype: object

In [47]:
dfs = df_scan.copy()

In [62]:
dfs.columns

Index(['Id', 'Dateiname', 'Dateidatum', 'Xmlzeile', 'Scanzeile', 'Suchzeile',
       'Treffer', 'Artikelzeile', 'TeilnehmerDurchlesewert', 'Bewertung',
       'Erfassungsdatum', 'Erfassungsuhrzeit', 'Abgeschlossen',
       'WellenteilnahmenId', 'TeilartikelVeroeffentlichungId',
       'Erscheinungsdatum', 'XPosition', 'YPosition', 'Seitennummer'],
      dtype='object')

In [87]:
def create_bw(df):
    
    cols = ["ArtikelId", "BW", "DW", "Datum"]
    new_df = pd.DataFrame(columns=cols)
    
    for date in df.Erscheinungsdatum.unique():
       
        df_ = df[df["Erscheinungsdatum"]==date]
        
        all_reader = df_.WellenteilnahmenId.nunique()
    
        liste_all_articles = df_.TeilartikelVeroeffentlichungId.unique()
        
        for ari in liste_all_articles: 
            df_ari = df_[df_["TeilartikelVeroeffentlichungId"]==ari]
            anzahl_leser_artikel = df_ari.shape[0]
            
            bw = anzahl_leser_artikel / all_reader * 100
            dw = df_ari.TeilnehmerDurchlesewert.mean()
            
        
            new_ser = pd.Series([ari, bw, dw, date], index=cols)

            new_df = new_df.append(new_ser, ignore_index=True)

    return new_df

dd = create_bw(df_all)    

AttributeError: 'DataFrame' object has no attribute 'Erscheinungsdatum'

In [125]:
def create_bw_2(df):
    
    cols = ["SplitId", "BW", "DW", "AusgabeErscheinungId", "Leser_Ausgabe", "Leser_Artikel"]
    new_df = pd.DataFrame(columns=cols)
    
    for aus in df.AusgabeErscheinungId.unique():
       
        df_ = df[df["AusgabeErscheinungId"]==aus]
        
        all_reader = df_.WellenteilnahmenId.nunique()
    
        liste_all_articles = df_.TeilartikelId.unique()
        
        for ari in liste_all_articles: 
            df_ari = df_[df_["TeilartikelId"]==ari]
            anzahl_leser_artikel = df_ari.shape[0]
            
            bw = anzahl_leser_artikel / all_reader * 100
            dw = df_ari.TeilnehmerDurchlesewert.mean()
            
        
            new_ser = pd.Series([ari, bw, dw, aus, all_reader, anzahl_leser_artikel], index=cols)

            new_df = new_df.append(new_ser, ignore_index=True)

    return new_df

dd = create_bw_2(df_all)    

In [117]:
test = dd[dd["AusgabeErscheinungId"]==236]
test[test["ArtikelId"]==15245]
#dd.rename(columns={"ArtikelId":"SplitId"}, inplace=True)

In [173]:
dd.to_csv("df_teilnehmer.csv", index=False)

In [ ]:
def create_bw_3(df):
    
    cols = ["SplitId", "BW", "DW", "AusgabeErscheinungId", "Leser_Ausgabe", "Leser_Artikel"]
    new_df = pd.DataFrame(columns=cols)
    
   
       
        df_ = df[df["AusgabeErscheinungId"]==aus]
        
        all_reader = df_.WellenteilnahmenId.nunique()
    
        liste_all_articles = df_.TeilartikelId.unique()
        
        for ari in liste_all_articles: 
            df_ari = df_[df_["TeilartikelId"]==ari]
            anzahl_leser_artikel = df_ari.shape[0]
            
            bw = anzahl_leser_artikel / all_reader * 100
            dw = df_ari.TeilnehmerDurchlesewert.mean()
            
        
            new_ser = pd.Series([ari, bw, dw, aus, all_reader, anzahl_leser_artikel], index=cols)

            new_df = new_df.append(new_ser, ignore_index=True)

    return new_df

dd = create_bw_2(df_all)    

In [131]:
df_all

,Id,TeilartikelVeroeffentlichungenId,WellenteilnahmenId,TeilnehmerDurchlesewert,Bewertung,ScanAuswertungId,TeilartikelId,AusgabeErscheinungId,XPosition,YPosition
0,1,4790,494,88.235291,0.0,34,1521,27,None,None
1,2,4806,494,78.378380,0.0,36,1540,27,None,None
2,3,4797,494,71.974525,0.0,32,1628,27,None,None
3,4,4795,494,99.149376,0.0,43,1570,27,None,None
4,5,4789,494,89.199997,0.0,41,1503,27,None,None
...,...,...,...,...,...,...,...,...,...,...
476602,482294,60519,364,28.119801,0.0,729214,19274,297,None,None
476603,482295,60591,364,99.370110,0.0,729215,19301,297,None,None
476604,482296,60688,364,99.511772,0.0,729216,19327,297,None,None
476605,482297,60523,364,99.486412,0.0,729217,19273,297,None,None


In [110]:
df_m_nk = pd.merge(dfnk[["ArtikelId", "SplitId", "Ressort", "Ressort_neu3", "LW", "BW", "DW"]],\
                dd[["ArtikelId" ,"BW", "DW"]], on="ArtikelId")

In [126]:
df_m_k = pd.merge(df_k[["SplitId", "Ressort", "Ressort_neu3", "LW", "BW", "DW"]],\
                dd[["SplitId" ,"BW", "DW"]], on="SplitId")

In [127]:
df_m_k

,SplitId,Ressort,Ressort_neu3,LW,BW_x,DW_x,BW_y,DW_y
0,5617,Lokales,Landkreis Starnberg,10.25,14.52,70.62,14.516129,70.621753
1,7222,Lokales,Landkreis Starnberg,16.12,17.65,91.33,17.391304,91.329286
2,5922,Lokales,Landkreis Starnberg,17.95,22.95,78.22,22.950820,78.221779
3,5924,Lokales,Landkreis Starnberg,7.13,13.11,54.34,13.114754,54.343622
4,5927,Lokales,Landkreis Starnberg,9.96,13.11,75.96,13.114754,75.961536
...,...,...,...,...,...,...,...,...
35773,14323,Schule und Hochschule,Schule und Hochschule,7.97,8.94,89.13,8.928571,95.444917
35774,14323,Schule und Hochschule,Schule und Hochschule,7.97,8.94,89.13,5.263158,57.415255
35775,14323,Schule und Hochschule,Schule und Hochschule,7.97,8.94,89.13,15.873016,86.430086
35776,14323,Schule und Hochschule,Schule und Hochschule,7.97,8.94,89.13,1.666667,96.927963


In [128]:
df_m_k.to_clipboard(decimal=",", index=False)

In [130]:
dd.SplitId.nunique()

12937

In [113]:
df_m_nk[df_m_nk["SplitId"]==15245]

,ArtikelId,SplitId,Ressort,Ressort_neu3,LW,BW_x,DW_x,BW_y,DW_y
24050,47897,15245,Sport,Sport,3.779401,4.132231,91.461499,4.132231,91.461499


In [ ]:
te = df_nk[df_nk[""]]

In [ ]:
df_m = 

In [107]:
dfnk.columns

Index(['SplitId', 'ArtikelId', 'Artikel', 'ErscheinungsId', 'AusgabeId',
       'Datum', 'Seite', 'Ausgabeleser', 'Artikelleser', 'BW', 'DW', 'LW',
       'Seitentitel', 'Platzierungen', 'Darstellungsformen', 'Beschreibung',
       'Handlungsorte', 'Akteure', 'Themen', 'Ausgabe', 'Ressort', 'Layerlink',
       'Ausgabeteil', 'Kumulierung', 'Wellennummer', 'Dachzeile', 'Wochentag',
       'Worte', 'Seitenreihung', 'Optik', 'Ressort_neu', 'Ressort_neu2', 'DF',
       'Sortierung_Ressort_neu.1', 'PL.1', 'PL_Sortierung', 'Wortzahl_nk',
       'Optik_nein', 'Wortzahl_neu_nk', 'Platzierung_neu_nk',
       'Platzierung_neu_Sortierung_nk', 'Ressort_neu3'],
      dtype='object')

In [104]:
dfnk = pd.read_clipboard(decimal=",")

<ipython-input-104-63b93aa3c438>:1: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  dfnk = pd.read_clipboard(decimal=",")


In [106]:
dfnk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35723 entries, 0 to 35722
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SplitId                        35723 non-null  int64  
 1   ArtikelId                      35723 non-null  int64  
 2   Artikel                        35709 non-null  object 
 3   ErscheinungsId                 35723 non-null  int64  
 4   AusgabeId                      35723 non-null  int64  
 5   Datum                          35723 non-null  object 
 6   Seite                          35723 non-null  int64  
 7   Ausgabeleser                   35723 non-null  int64  
 8   Artikelleser                   35723 non-null  int64  
 9   BW                             35723 non-null  float64
 10  DW                             35723 non-null  float64
 11  LW                             35723 non-null  float64
 12  Seitentitel                    35723 non-null 

## Teilnehmer 

In [75]:
dd["ArtikelId"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48959 entries, 0 to 48958
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ArtikelId  48859 non-null  float64
 1   BW         48959 non-null  float64
 2   DW         48154 non-null  float64
 3   Datum      48959 non-null  object 
dtypes: float64(3), object(1)
memory usage: 1.5+ MB


In [99]:
t = df_nk[df_nk["ArtikelId"]==56146]
t[['Artikel-Blickwert (Erscheinung) in %',
       'Artikel-Durchlesewerte (Erscheinung) in %',
       'Artikel-Lesewert (Erscheinung) in %']]

,Artikel-Blickwert (Erscheinung) in %,Artikel-Durchlesewerte (Erscheinung) in %,Artikel-Lesewert (Erscheinung) in %
30240,3.072626,69.936914,2.1489


In [121]:
df_k = pd.read_clipboard(decimal=",")

In [122]:
df_k

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Wortzahl,Ressort_neu,Sortierung_Ressort_neu.1,Ressort_neu2,Sortierung_Ressort_neu.2,Wortzahl_neu,Platzierung_neu,Platzierung_neu_Sortierung,Ressort_neu3,Wochenende
0,5617,17831,Motorradfahrerin stößt mit Kleinbus zusammen,90,5,13.04.2021,39,62,9,14.52,...,unter 100,Landkreis Starnberg,210.0,Landkreis Starnberg,210.0,unter 500,Einspalter,3,Landkreis Starnberg,nein
1,7222,22808,Handwerker unterwegs mit zwei Promille,115,5,19.04.2021,35,68,12,17.65,...,unter 100,Landkreis Starnberg,210.0,Landkreis Starnberg,210.0,unter 500,Einspalter,3,Landkreis Starnberg,nein
2,5922,18655,Berger Rathausspitze negativ getestet,95,5,14.04.2021,35,61,14,22.95,...,unter 100,Landkreis Starnberg,210.0,Lok 1 Starnberg,187.0,unter 500,Einspalter,3,Landkreis Starnberg,nein
3,5924,18682,Haltestelle in Tutzing verlegt,95,5,14.04.2021,37,61,8,13.11,...,unter 100,Landkreis Starnberg,210.0,Landkreis Starnberg,210.0,unter 500,Einspalter,3,Landkreis Starnberg,nein
4,5927,18696,"Kreativ, mobil und umweltschonend",95,5,14.04.2021,37,61,8,13.11,...,unter 100,Landkreis Starnberg,210.0,Landkreis Starnberg,210.0,unter 500,Einspalter,3,Landkreis Starnberg,nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11316,6061,18822,Geschenk aus Malta,96,1,15.04.2021,32,381,30,7.87,...,unter 100,Reise,150.0,Reise,150.0,unter 500,Einspalter,3,Reise,nein
11317,7954,24844,Neue Kreuzfahrten,126,1,22.04.2021,32,405,17,4.20,...,unter 100,Reise,150.0,Reise,150.0,unter 500,Einspalter,3,Reise,nein
11318,7988,24887,Portugal öffnet,126,1,22.04.2021,32,405,33,8.15,...,unter 100,Reise,150.0,Reise,150.0,unter 500,Einspalter,3,Reise,nein
11319,14280,45021,Lehrer als Blitzableiter,221,1,17.05.2021,13,358,36,10.06,...,unter 100,Schule und Hochschule,NaN,Schule und Hochschule,NaN,unter 500,Einspalter,3,Schule und Hochschule,nein


In [136]:
# nur WellenteiehamenId von Nutzern, deren Nutzer Id mit 2014 anfange

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [142]:
sql_teilnehmer = '''

SELECT te.Teilnehmernummer, we.Id as WellenteilnahmenId, we.WellenId, we.Abgebrochen

FROM Teilnehmer te
JOIN Wellenteilnahmen we
on te.Id = we.TeilnehmerId

'''

In [143]:
df_te = pd.read_sql(sql_teilnehmer, cnxn)

In [176]:
df_te.to_pickle("teilnehmer.pkl")

In [168]:
df_teil = df_te[df_te["Abgebrochen"]==0]

In [169]:
#df_teil = df_te.copy()

In [171]:
df_tt= df_teil[df_teil["Teilnehmernummer"].str[:4]=="2014"]
wid = df_tt.WellenteilnahmenId.unique()

In [161]:
df_all.WellenteilnahmenId.nunique()

599